## Wav2Lip
**[Wav2Lip](https://arxiv.org/pdf/2008.10010.pdf)** 是一种基于对抗生成网络的由语音驱动的人脸说话视频生成模型。如下图所示，Wav2Lip的网络模型总体上分成三块：生成器、判别器和一个预训练好的Lip-Sync Expert组成。网络的输入有2个：任意的一段视频和一段语音，输出为一段唇音同步的视频。生成器是基于encoder-decoder的网络结构，分别利用2个encoder: speech encoder, identity encoder去对输入的语音和视频人脸进行编码，并将二者的编码结果进行拼接，送入到 face decoder 中进行解码得到输出的视频帧。判别器Visual Quality Discriminator对生成结果的质量进行规范，提高生成视频的清晰度。为了更好的保证生成结果的唇音同步性，Wav2Lip引入了一个预预训练的唇音同步判别模型 Pre-trained Lip-sync Expert，作为衡量生成结果的唇音同步性的额外损失。

### Lip-Sync Expert
Lip-sync Expert基于 **[SyncNet](https://www.robots.ox.ac.uk/~vgg/publications/2016/Chung16a/)**，是一种用来判别语音和视频是否同步的网络模型。如下图所示，SyncNet的输入也是两种：语音特征MFCC和嘴唇的视频帧，利用两个基于卷积神经网络的Encoder分别对输入的语音和视频帧进行降纬和特征提取，将二者的特征都映射到同一个纬度空间中去，最后利用contrastive loss对唇音同步性进行衡量，结果的值越大代表越不同步，结果值越小则代表越同步。在Wav2Lip模型中，进一步改进了SyncNet的网络结构：网络更深；加入了残差网络结构；输入的语音特征被替换成了mel-spectrogram特征。

## 1. 环境的配置
- `建议准备一台有显卡的linux系统电脑，或者可以选择使用第三方云服务器` 
- `Python 3.6 或者更高版本` 
- ffmpeg: `sudo apt-get install ffmpeg`
- 必要的python包的安装，所需要的库名称都已经包含在`requirements.txt`文件中，可以使用 `pip install -r requirements.txt`一次性安装. 
- 在本实验中利用到了人脸检测的相关技术，需要下载人脸检测预训练模型：Face detection [pre-trained model](https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth) 并移动到 `face_detection/detection/sfd/s3fd.pth`文件夹下. 

In [ ]:
!pip install -r requirements.txt

In [ ]:
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O face_detection/detection/sfd/s3fd.pth

## 2. 数据集的准备及预处理

**LRS2 数据集的下载**  
实验所需要的数据集下载地址为：<a href="http://www.robots.ox.ac.uk/~vgg/data/lip_reading/lrs2.html">LRS2 dataset</a>，下载该数据集需要获得BBC的许可，需要发送申请邮件以获取下载密钥，具体操作详见网页中的指示。下载完成后对数据集进行解压到本目录的`mvlrs_v1/`文件夹下，并将LRS2中的文件列表文件`train.txt, val.txt, test.txt` 移动到`filelists/`文件夹下，最终得到的数据集目录结构如下所示。
```
data_root (mvlrs_v1)
├── main, pretrain (我们只使用main文件夹下的数据)
|	├── 文件夹列表
|	│   ├── 5位以.mp4结尾的视频ID
```
**数据集预处理**
数据集中大多数视频都是包含人的半身或者全身的画面，而我们的模型只需要人脸这一小部分。所以在预处理阶段，我们要对每一个视频进行分帧操作，提取视频的每一帧，之后使用`face detection`工具包对人脸位置进行定位并裁减，只保留人脸的图片帧。同时，我们也需要将每一个视频中的语音分离出来。

In [ ]:
!rm -rf ../LSR2/demo
!mkdir -p ../LSR2/demo
!cp -r ../LSR2/main/553* ../LSR2/demo/

In [ ]:
!python generate_hq_videos.py --data_root ../LSR2/demo/

In [ ]:
# !rm -rf ../LSR2/lrs2_preprocessed_288x288
# !python preprocess.py --data_root "../LSR2/main_hq" --preprocessed_root "../LSR2/lrs2_preprocessed_288x288" --batch_size 32 --ngpu 4
!rm -rf ../LSR2/lrs2_preprocessed_288x288-demo
!python preprocess.py --data_root "../LSR2/demo" --preprocessed_root "../LSR2/lrs2_preprocessed_288x288-demo" --batch_size 128

预处理后的`lrs2_preprocessed/`文件夹下的目录结构如下
```
preprocessed_root (lrs2_preprocessed)
├── 文件夹列表
|	├── 五位的视频ID
|	│   ├── *.jpg
|	│   ├── audio.wav
```

获取对应的文件列表并更新到filelists/train.txt和filelists/eval.txt。只保存对应的视频名称即可。代码可以参考，对视频样本重命名并生成对应的命名列表，此处视频文件数量过少<2，会报错：

In [ ]:
!python generate_filelists.py --data_root ../LSR2/lrs2_preprocessed_288x288-demo

Training the expert discriminator

In [ ]:
!python color_syncnet_train.py --data_root ../LSR2/lrs2_preprocessed_288x288-demo/ --checkpoint_dir ./savedmodel 
# !python color_syncnet_train.py --data_root ../LSR2/lrs2_preprocessed_288x288/ --checkpoint_dir ./savedmodel 
# !python color_syncnet_train.py --data_root ../LSR2/lrs2_preprocessed_288x288/ --checkpoint_dir ./savedmodel --checkpoint_path ./savedmodel/checkpoint_step000032000.pth 
# --checkpoint_path ./checkpoints/lipsync_expert.pth

执行如下命令，开始训练：

In [ ]:
!python wav2lip_train.py --data_root ../LSR2/lrs2_preprocessed_288x288-demo --checkpoint_dir ./savedmodel --syncnet_checkpoint_path ./savedmodel/checkpoint_step000000001.pth 
# !python wav2lip_train.py --data_root ../LSR2/lrs2_preprocessed_288x288 --checkpoint_dir ./savedmodel --syncnet_checkpoint_path ./savedmodel/checkpoint_step000032000.pth 
# --checkpoint_path ./checkpoints/wav2lip.pth

In [ ]:
!python hq_wav2lip_train.py --data_root ../LSR2/lrs2_preprocessed_288x288-demo --checkpoint_dir ./savedmodel --syncnet_checkpoint_path ./savedmodel/checkpoint_step000000001.pth 
# !python hq_wav2lip_train.py --data_root ../LSR2/lrs2_preprocessed_288x288 --checkpoint_dir ./savedmodel --syncnet_checkpoint_path ./savedmodel/checkpoint_step000032000.pth 

In [ ]:
!python wloss_hq_wav2lip_train.py --data_root ../LSR2/lrs2_preprocessed_288x288-demo/ --checkpoint_dir ./savedmodel --syncnet_checkpoint_path ./savedmodel/checkpoint_step000000001.pth
# !python wloss_hq_wav2lip_train.py --data_root ../LSR2/lrs2_preprocessed_288x288/ --checkpoint_dir ./savedmodel --syncnet_checkpoint_path ./savedmodel/checkpoint_step000050000.pth 
# --checkpoint_path ./checkpoints/wav2lip.pth

In [ ]:
%%time
!python inference.py --checkpoint_path ./savedmodel/wav2lip_checkpoint_step000093000.pth --face ../videos/97.mp4 --audio ../videos/test.wav
